`TODO`:
- [x] get classes working w toy models 
- [ ] integrate `expt1_oob_models.py` into this or additional nb
- [ ] move clf classes to module 
- [ ] move the mwe's below into smthg else (maybe just in docstrings?)
- [ ] figger out clean way to do imports for class module (tricky) 
- [ ] reorganize + simplify evaluation interface 
- [ ] figger out clever way to avoid prepping text repeatedly 
- [ ] determine whether TypeC even needs a separate class (def needs minimal text prep)
- [ ] improve preprocessing interface 
- [ ] add sub-expt w train on ~70% and eval on length subsets 

<hr><hr>
<br><br>

## Experiment 1
#### Document length and classification accuracy
<hr>

###### Part 1: Define experiment 
1. define space of classifiers (and hypers) to consider 
2. import classes for managing arbitrary `sklearn` and `keras` models 
3. write model evaluation function 
4. write performance curve flattener 

###### Part 2: Prepare data (`TODO` -- reorganize splits)
1. import text preprocessing functions/classes (`TODO` -- decide which + put in module)
2. load and preprocess text (need >1 format for diff clf styles)
3. split data into length subsets 
4. set aside 30% of data for evaluation, stratified by length and label

###### Part 3: Conduct experiment 
1. for each train subset, get 5-fold crossval F1 for each clf
2. for each fit, generate and save preds on evaluation set 

###### Part 4: Evaluate results
1. plot the crossval F1 scores across subsets and clfs
2. plot performance on validation set for each clf and subset 


<br>
#### Part 1: Define experiment
<hr>

##### 1.1 Define space of classifiers 

We will consider the following set of classification strategies:

- `Type A:` Non-NN: 
    - Multinomial Naive Bayes
    - Support Vector Machine
- `Type B:` Feed-forward NN classifiers: 
    - Multilayer Perceptron
    - Convolutional NN
- `Type C:` Recurrent NN classifiers: 
    - LSTM Network 
    - Bi-directional RNN

> *Note:* `Type C` algorithms take sequence data as input (padded token sequences), whereas `Type A` and `Type B` take DTM-type structures as input (vectorized sequences).  

##### 1.2 Import wrapper class for each model type 

In [1]:
from modules.expt1_classes import TypeA  # for `sklearn` classifiers 
from modules.expt1_classes import TypeB  # for `keras` Sequential models

Using TensorFlow backend.


##### 1.3 Write function to evaluate performance 

In [2]:
# will use f1_score directly in the below  
from sklearn.metrics import f1_score

# utility func to convert a probability into a binary prediction 
def prob_to_binary(prob, threshold=.5, ret_type=int):
  assert 0 <= prob <= 1
  assert 0 <= threshold <= 1
  return ret_type(prob > threshold)

# TODO: is this necessary? confusing?? etc. 
def score_preds(clf, test_X, test_y, metric, pred_transformer=lambda x: x):
  test_preds = [pred_transformer(y) for y in clf.predict(test_X)]
  return metric(test_y, test_preds)

##### 1.4 Write function to flatten results dicts

In [3]:
# TODO 

<br>
#### Part 2: Prepare data
<hr>

##### 2.1 Import text preprocessors 

In [4]:
# TODO 

##### 2.2 Load and preprocess text and labels 

In [5]:
# TODO

##### 2.3 Split data into length bins (defined in the `.csv`)

In [6]:
# TODO 

##### 2.4 Set aside 30% of data for evaluation, stratified by length and label

In [7]:
# TODO

<br>
#### Part 3: Conduct experiment
<hr>

##### 3.0 Check that classes work as designed

In [8]:
# TODO [need to modify this + integrate into secs 2.2, 2.3, 2.4 above
import pandas as pd

data_file = 'data/imdb_decoded.csv'
vocab_n = 10000


dat = pd.read_csv(data_file)

train = dat[dat.subset=='train']
test = dat[dat.subset=='test']

# TODO: encode labels for nn mods (not nec but good for exposition)
train_text, train_labels = train.text, train.label
test_text, test_labels = test.text, test.label


print(f'shape of full data: {dat.shape[0]}x{dat.shape[1]}')
print('first few rows:')
display(dat.head(2))

shape of full data: 50000x5
first few rows:


,subset,length,length_bin,label,text
0,train,217,2,1,this film was just brilliant casting location ...
1,train,188,2,0,big hair big boobs bad music and a giant safet...


In [9]:
### TypeA example 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# specify classifier and associated hypers 
skl_clf = MultinomialNB
skl_clf_params = {'alpha': .9, 'fit_prior': True}

# preprocess text 
skl_token_params = {'ngram_range': (1,1), 'min_df': 2, 'max_features': vocab_n}
vectorizer = CountVectorizer(**skl_token_params)
train_dtm = vectorizer.fit_transform(train_text)
test_dtm = vectorizer.transform(test_text)

# instantiate and train classifier 
classifier = TypeA(skl_clf, **skl_clf_params)
classifier.train(train_dtm, train_labels)

# generate predictions on holdout set 
predsA = classifier.predict(test_dtm)

In [10]:
### TypeB example
from keras import models
from keras import layers
from keras.preprocessing.text import Tokenizer

# specify classifier and associated hypers 
keras_model = models.Sequential
layers_list = [layers.Dense, layers.Dense, layers.Dense]
kwargs_list = [dict(units=16, activation='relu', input_shape=(vocab_n, )), 
               dict(units=16, activation='relu'), 
               dict(units=1,  activation='sigmoid')]
compile_params = dict(optimizer='rmsprop', 
                      loss='binary_crossentropy', 
                      metrics=['accuracy'])
train_params = dict(valset_prop=.3, epochs=4, batch_size=500)


# preprocess text 
tokenizer = Tokenizer(num_words=vocab_n)
tokenizer.fit_on_texts(train_text)
train_encoded = tokenizer.texts_to_sequences(train_text)
test_encoded = tokenizer.texts_to_sequences(test_text)
train_dtm = tokenizer.sequences_to_matrix(train_encoded, mode='count')
test_dtm = tokenizer.sequences_to_matrix(test_encoded, mode='count')
# print(f'dims of train input, targets: {train_dtm.shape}, {train_labels.shape}')
# print(f'dims of test  input, targets: {test_dtm.shape}, {test_labels.shape}')


# instantiate, compile, and train classifier 
# (make sure inputs and targets are appropriately preprocessed!!!) 
neural_net = TypeB(keras_model, layers_list, kwargs_list)
neural_net.compile_model(**compile_params)
neural_net.train(train_dtm, train_labels, **train_params)


# generate predictions on holdout set 
predsB = neural_net.predict(test_dtm, pred_postprocessor=prob_to_binary)

Train on 17500 samples, validate on 7500 samples
Epoch 1/4
 - 3s - loss: 0.5318 - acc: 0.7705 - val_loss: 0.4058 - val_acc: 0.8645
Epoch 2/4
 - 3s - loss: 0.3336 - acc: 0.8913 - val_loss: 0.3136 - val_acc: 0.8848
Epoch 3/4
 - 3s - loss: 0.2538 - acc: 0.9136 - val_loss: 0.2990 - val_acc: 0.8909
Epoch 4/4
 - 3s - loss: 0.2037 - acc: 0.9323 - val_loss: 0.2864 - val_acc: 0.8892


In [11]:
### compare performance of TypeA and TypeB (not real expt)
print('  >> TypeA test f1:', round(f1_score(test_labels, predsA), 3))
print('  >> TypeB test f1:', round(f1_score(test_labels, predsB), 3))

  >> TypeA test f1: 0.812
  >> TypeB test f1: 0.884


##### 3.1 Get 5-fold crossval F1 for each clf across train subsets


##### 3.2 Generate predictions on evaluation set

##### 3.3 Train on random 70% subset, evaluate across subsets on remaining 30%

<br>
#### Part 4: Evaluate results
<hr>

##### 4.1 Plot CV scores across subsets and clfs

##### 4.2 Plot performance on holdout set

<br><br><br><br><br>
<hr><hr>
### sqrache areyaya

In [12]:
%%script false 
### this is 80 chars long, just fyi ##########################################
##############################################################################

Couldn't find program: 'false'


In [13]:
%%script false 

import numpy as np

def boosh(x, **kwargs):
  print(f'x is {x}...')
  return np.random.normal(**kwargs)

def boosh2(f_list, kwargs_list):
  assert len(f_list) == len(kwargs_list)
  for f, kwargs in zip(f_list, kwargs_list):
    val = f(**kwargs)
    print(f'{f.__name__} applied to kwargs: {val}')


# kw = dict(loc=1, scale=2, size=3)
# np.random.normal(**kw)
# boosh(3, **kw)

# np.random.exponential(scale=1.0, size=5)
# np.random.chisquare(df, size=1)


fs = [np.random.normal, np.random.exponential, np.random.chisquare]
kwargss = [dict(loc=1, scale=2, size=3), 
           dict(scale=2.0, size=2), 
           dict(df=2, size=1)]
np.random.seed(6933)
boosh2(fs, kwargss)

Couldn't find program: 'false'


###### sketch of expt1 format 
<hr>

```
# here is some pseudocode illustrating the approach 

dat = dataset

length_bins = [0,1,2,3,4]

results = {}

# for each subset defined by length 
for bin in length_bins:
  dat_subset = dat[length_bin==length_bin]
  train_text, train_labels = dat.text[subset=='train'],  dat.label[subset=='train']
  test_text,  test_labels  = dat.text[subset=='test'], dat.label[subset=='test']
  
  # preprocess the text, separately for A versus B(?!)
  train_dtmA = A_style_transformer(train_text)
  train_dtmB = B_style_transformer(train_text)
  test_dtmA = A_style_transformer(test_text)
  test_dtmB = B_style_transformer(test_text)
  
  # specify necessary hypers for each clf (maybe do elsewhere?!) 
  hypersA = {some-hypers-for-clf-A}
  hypersB = {some-hypers-for-clf-B}
  
  # train classifiers 
  clfA = Classifier_TypeA(hypersA).train(train_dtmA, train_labels)
  clfB = Classifier_TypeA(hypersB).train(train_dtmB, train_labels)
  
  # generate predictions 
  predsA = clfA.predict(test_dtmA)
  predsB = clfB.predict(test_dtmB)
  
  # score the preds 
  results['bin'+str(bin)] = {'A_score': f1_score(test_labels, predsA), 
                             'B_score': f1_score(test_labels, predsB)}
  
    
# postprocess the results dict 
results_df = dataframeify(results)
results_df.to_csv('expt1_results.csv')

# then go make nice plots in an R Markdown doc :p 
```
